In [ ]:
pip install galois

In [ ]:
# Needed libraries
import secrets
import math
import numpy as np
import hashlib
import galois

# Generate private seed

In [ ]:
private_seed = secrets.token_bytes(32)
print(private_seed)

b'\xcc\xee\xf8*l_/\x1cP\xbe1\xea\x12\xd9/PY\xd4b\xe8m\xc0&\x7f\xb4F\x05\x05h\xe8\x178'


# SHAKE A.K.A function H (Squeeze public_seed and T)

In [ ]:
v = 197
m = 57
lvl = 1

## SHAKE using hashlib

In [ ]:
if(lvl==1):
  h_shake = hashlib.shake_128(private_seed)
else:
  h_shake = hashlib.shake_256(private_seed)
private_sponge = h_shake.digest(32+math.ceil(m/8)*v)
print(private_sponge)

b'\xc3\xb6\x83>\xe2\x906\x95\xb8\xed#6\x9a\xac\x94.\xb9\xa2",\xb9\xe7m\x15\xa9U\x82=\xf2\xb6\xf6J\x8b\x9aa\xaa\xbe]Y\xca\xb7\x011\x85\x9dqw[\xa85W\xb0\xc3\xfc]\x97\x0e\x0b\x04\xdc\x84\x7f\x8e5\xa1\x8d\xcfD#Q7r\x1c\x17\xcc\x07\xd5\x90y0O\xbc\xd9\x96SJ\xe1\xc8\x90\x1b\x02\xc96\x97\xd0\x8d\x15!\x08a\xccUf\xb0wM[\x00T8y\xd3\x129\xb7$\x83\x17\x8a\xb7\xd5S{;0\xa7<\xa4\x07\xb5\x83\xba*\x07_\xfe\x05\xb7)\x82\xd9\x7fB\x1d\x15e\x838\x9aK\xf84\xaa{s\xa6\x82\x83\xe1;\xbc\xdf\xf0\xfa\xcf\xb2\x15\x84_\xcb@\n\x0fb4\x8f;\xfd=\xf8Y]\xf3o\xcbOJh1\xd1$=\x0ej\xe5J_<%;\xbfFaX\x18x\xec\x97?\xb1\xa2\xe7\xae\x02\x1c\x81\x07\xc3*\xee B\x16\xd6\xd3C\x91\x15t\xb5\xc2DSW\xd5tD%\xe0\x1aH\x9f\x1d\xc5\x11cS`\x83;\xbf\xb73\xecG\xd0\x1a\xb7\xd9B.fS\x06!\xfco\xf2\xd4i\xe2\xc8\xbb\xb7gU\x06\xfe\x1e\xbet\xf3\xe9\xdf\xe0\x1d\xbe[\x9e\x9b\x84\xa5\x19\xca\xbbka\x18\xcd*F\xff\xa59\xd6u\x0bEig7\xabZ\xbc\xc8\xe4^\xf5O\x9eI\xd3\xa8\xba\xd2\xd2\xc3K\'\xbc\xddV-\x11\xe9qo\xde\xcc\xdf\x12\x92C\x9a\x8d/\x9e\xb3\xeai\x03h\xe5V@"\xfb

#Extract public seed

In [ ]:
public_seed = private_sponge[:32]
print(public_seed)

b'\xc3\xb6\x83>\xe2\x906\x95\xb8\xed#6\x9a\xac\x94.\xb9\xa2",\xb9\xe7m\x15\xa9U\x82=\xf2\xb6\xf6J'


#Extract T

In [ ]:
def int8_to_bits(intByte:int)->list:
  """Get the first 8 bits of an integer as an array.

  The function takes one integer byte, extracts it's bits
  and returns an array containing the bits starting from
  the most significant bit.

  Parameters
  ----------
  intByte : int
    The byte, labeled as an int by python.

  Returns
  -------
  list
      An array with the first 8 bits of the integer
      starting from the most significant one.
  """
  bits_data = bin(intByte)[2:]
  return [int(i) for i in bits_data.zfill(8 * ((len(bits_data) + 7) // 8))]

In [ ]:
def tMatrixRowGen(byteString:bytes,neededBits:int)->list:
  """Generate one row of the T matrix.

  The function takes one byte-string and extracts the specified
  ammount of bits (neededBits) to fill one row. When the ammount
  of bits doesn't fit completely in a byte, the most significant
  bits of the last byte are ignored.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the row.
  neededBits : int
    The ammount of bits a row needs to have.

  Returns
  -------
  list
      An array representing one row of the T matrix
  """
  row = []
  while(neededBits>8):
    row+=int8_to_bits(byteString[0])
    byteString = byteString[1:]
    neededBits = neededBits-8
  if(neededBits>0):
    row+=int8_to_bits(byteString[0])[-neededBits:]
  return row

def tMatrixGenerator(byteString:bytes,v:int,m:int)->np.ndarray:
  """Generate the T matrix.

  The function takes one byte-string and generates v rows
  each one with m bits extracted sequentially from a byteString
  using the tMatrixRowGen() function.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the matrix.
  v : int
    The ammount rows of the matrix.
  m : int
    The ammount columns of the matrix.

  Returns
  -------
  list
      An array representing the T matrix
  """
  mdivided8 =math.ceil(m/8)
  matrix = []
  for i in range(v):
    matrix.append(tMatrixRowGen(byteString,m))
    byteString = byteString[mdivided8:]
  return np.array(matrix)

In [ ]:
T_base = private_sponge[32:]
T = tMatrixGenerator(T_base,v,m)
print(T)

[[1 0 0 ... 0 1 0]
 [1 0 1 ... 1 1 1]
 [1 0 1 ... 0 1 1]
 ...
 [1 0 0 ... 1 0 1]
 [0 1 0 ... 1 0 1]
 [0 0 0 ... 1 0 0]]


# SHAKE (squeeze public map)

In [ ]:
def subMatrixGenerator(byteString:bytes)->np.ndarray:
  """Generate a part of the L or Q1 matrix.

  The function takes one byte-string and returns one 16 row part
  of the L or Q1 matrix. It must be noted that the rows are generated as columns
  at first and then are transposed.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the matrix.

  Returns
  -------
  np.ndarray
      A matrix
  """

  matrix = [int8_to_bits(byteString[i*2])+int8_to_bits(byteString[i*2+1]) for i in range(len(byteString)//2)]

  return np.array(matrix).T

In [ ]:
n = m+v
g_equation = 2+2*n+v*(v+1)+2*v*m
mdivided16 = math.ceil(m/16)
print(n)
print(mdivided16)

#first g function iteration
if(lvl==1):
  g_shake = hashlib.shake_128(public_seed+bytes.fromhex('00'))
else:
  g_shake = hashlib.shake_256(public_seed+bytes.fromhex('00'))
public_sponge = g_shake.digest(g_equation)
C_base=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
public_sponge = public_sponge[2:]
# L and Q1 have more than one dimension, so they are initilized differently
L_base = subMatrixGenerator(public_sponge[:2*n])
public_sponge = public_sponge[2*n:]
Q1_base = subMatrixGenerator(public_sponge)

#following g function iterations
if(lvl==1):
  for i in range(1,mdivided16):
    g_shake = hashlib.shake_128(public_seed+bytes.fromhex(f'0{i}'))
    public_sponge = g_shake.digest(g_equation)
    C_base+=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
    public_sponge = public_sponge[2:]
    L_base = np.concatenate((L_base,subMatrixGenerator(public_sponge[:2*n])))
    public_sponge = public_sponge[2*n:]
    Q1_base = np.concatenate((Q1_base,subMatrixGenerator(public_sponge)))
else:
  for i in range(1,mdivided16):
    g_shake = hashlib.shake_256(public_seed+bytes.fromhex(f'0{i}'))
    public_sponge = g_shake.digest(g_equation)
    C_base+=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
    public_sponge = public_sponge[2:]
    L_base = np.concatenate((L_base,subMatrixGenerator(public_sponge[:2*n])))
    public_sponge = public_sponge[2*n:]
    Q1_base = np.concatenate((Q1_base,subMatrixGenerator(public_sponge)))


254
4


### Obtaining C

In [ ]:
# we take the m bits (starting from the last generated bit) we need from
# the total of bits generated
C = C_base[-m:]
C = np.array(C)
print(C)

[0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 1 0 1 1 1 1 0 0 1 0 1 1 0 1 0 1 1 1 1 1 1 0
 0 0 0 0 0 1 1 0 0 1 1 1 1 1 1 0 1 0 0 0]


### Obtaining L

In [ ]:
#We discard the upper rows to get the specified dimensions
L = L_base[-m:]
print(L)

[[0 1 0 ... 1 1 0]
 [1 1 1 ... 0 1 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 1 0 ... 1 0 1]
 [1 0 1 ... 1 1 0]
 [0 0 1 ... 1 1 1]]


### Obtaining $$Q_{1}$$

In [ ]:
#we discard the upper rows to get the specified dimensions
Q1=Q1_base[-m:]
print(Q1)

[[0 0 0 ... 1 1 1]
 [1 1 1 ... 1 0 1]
 [1 0 0 ... 0 1 1]
 ...
 [1 1 1 ... 1 0 0]
 [1 1 0 ... 0 0 0]
 [1 0 1 ... 1 1 0]]


# find $$Q_2$$

In [ ]:
def findPk1(k:int,q1:np.ndarray)->np.ndarray:
  """An implementation of the findPk1 algorithm of the LUOV specification.

  Parameters
  ----------
  k : int
    An integer between 1 and m.
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.

  Returns
  -------
  np.ndarray
      The v-by-v matrix representing the part of pk that is
      quadratic in the vinegar variables.
  """
  Pk1 = np.zeros([v,v],dtype=np.int8)
  column = 0
  for i in range(v):
    for j in range(i,v):
      Pk1[i,j]= q1[k,column]
      column+=1
    column+=m
  return Pk1

In [ ]:
def findPk2(k:int,q1:np.ndarray)->np.ndarray:
  """An implementation of the findPk2 algorithm of the LUOV specification.

  Parameters
  ----------
  k : int
    An integer between 1 and m.
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.

  Returns
  -------
  np.ndarray
      The v-by-m matrix representing the part of pk that is bilinear in the
      vinegar variables and the oil variables.
  """
  Pk2 = np.zeros([v,m],dtype=np.int8)
  column = 0
  for i in range(v):
    column += v-i
    for j in range(m):
      Pk2[i,j]= q1[k,column]
      column+=1
  return Pk2

In [ ]:
def findQ2(Q1:np.ndarray,T:np.ndarray)->galois.GF2:
  """An implementation of the findQ2 algorithm of the LUOV specification.

  Parameters
  ----------
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.
  t : np.ndarray
    A v-by-m matrix.

  Returns
  -------
  np.ndarray
      The second part of Macaulay matrix for quadratic part of P.
  """
  # we use galois to prevent the results of the operations from
  # being out of the finite field
  GF = galois.GF(2)
  T = GF(T)

  Q2 = GF(np.zeros([m,int(m*(m+1)/2)],dtype=np.int8))
  for k in range(m):
    Pk1 = GF(findPk1(k,Q1))
    Pk2 = GF(findPk2(k,Q1))
    Pk3 = -np.transpose(T)@Pk1@T+np.transpose(T)@Pk2
    column = 0
    for i in range(m):
      Q2[k,column]= Pk3[i,i]
      column += 1
      for j in range(i+1,m):
        Q2[k,column] = Pk3[i,j] + Pk3[j,i]
        column+=1
  return Q2

In [ ]:
Q2 = findQ2(Q1,T)
print(Q2)

[[0 1 0 ... 1 0 1]
 [1 1 0 ... 1 0 1]
 [0 0 1 ... 1 1 1]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 1 1 0]]


# Generate the public key

In [ ]:
# The first 32 bytes of the public key come from the public seed
public_key = public_seed

# The columns of Q2 are concatenated to get a sequence of bits for the public key
Q2_bits = Q2.flatten('F')
remaining_bits = len(Q2_bits)
Q2_bytes = b''

while(remaining_bits>8):
  byte = ''
  for i in Q2_bits[:8]:
    # Reverse the order of bits
    byte = str(Q2_bits[i]) + byte
  # Turn the bits into a byte and concatenate
  Q2_bytes+=int(byte,2).to_bytes(1,byteorder='big')
  remaining_bits -= 8
  Q2_bits = Q2_bits[8:]
if (remaining_bits>0):
  last_byte = ''
  for bit in Q2_bits:
    # We reverse the order of the
    last_byte = str(bit) + last_byte

  # Add the last byte
  Q2_bytes+=int(last_byte,2).to_bytes(1,byteorder='big')
print(Q2_bytes)

b'\xe2\x00"6\xee\xd6\xff\x96\x00\x82\xff\xd6\x12Z\x00J\x00\x12\xa2\xff\xc2\x00\x00\xaa\xff\x00\x00\xff\x00\x00\xfe:\x00\xfa&\xff\x00\x00\xd6\xff\xff\x00\x82\xc6\xfa>\x00\x00\x02\x00\x00\xfa\x00:\x00:"\xff\x00\xff\x8a\xfa>\xb2N\xfan\x92~\x00\xff\x00\xce\xff\x02\xff\xf2v\xff\xf2\xff~\x00\xde\x8e\xff\xff2~\xff\xff\x00\x9e\xffZ\xff\x00\x96\x00\xff\x00b\x00\xff\xff\xce:\x8e\x00\xff6\xfe\xff\xff\x00\x00Z\xda\xff\x00V\xff\x00v\xcej\x00j\xff\x00V\x0e\x00\xff\x00\x00\x00\xff\xe2.\x0e\x00\x00\x00\xaa\x00\xbe~\xaaj\x00\xff\xff"\xff\xe6V\xb6\xe6\x00\xba\xfab2\xff\xff\xc2\x12\xda\xff\xffb\xca\x16\xff&\xff2\x00\xff\x00\xf2\xffn&\xffRN\x00\x00Z\xff\x00\x00r\xff\x00\xff*\x00\x00\x12\xc6Z\x00\xffn\x92\xd2\xc2F\xff\xaa\xff\x06z\xff\x00fN\xd6\x00\x00b\xff\xd6\xff\x00\xff\x00\x00\xfa\x00v\x00\x00\xff\xff\x00\x1a\xa2\xff\xff\x12&\xff\xff\x92ZB\xba\xff\x00\xff\x00\xf6\xff\xd6\xea\xee\x86\x00\x00\xff\xc6\x00:\xd6\x8e^~\x00\x0e\xfe\xea\xff\x00\xff\xffV\xf6\x8e\xce\xff^\xff>\x00\xae\x00\n\x00\x00\xff\x00\xae\x

In [ ]:
# Add the bytes to the public key
public_key += Q2_bytes
print(public_key)
print(len(public_key))

b'\xc3\xb6\x83>\xe2\x906\x95\xb8\xed#6\x9a\xac\x94.\xb9\xa2",\xb9\xe7m\x15\xa9U\x82=\xf2\xb6\xf6J\xe2\x00"6\xee\xd6\xff\x96\x00\x82\xff\xd6\x12Z\x00J\x00\x12\xa2\xff\xc2\x00\x00\xaa\xff\x00\x00\xff\x00\x00\xfe:\x00\xfa&\xff\x00\x00\xd6\xff\xff\x00\x82\xc6\xfa>\x00\x00\x02\x00\x00\xfa\x00:\x00:"\xff\x00\xff\x8a\xfa>\xb2N\xfan\x92~\x00\xff\x00\xce\xff\x02\xff\xf2v\xff\xf2\xff~\x00\xde\x8e\xff\xff2~\xff\xff\x00\x9e\xffZ\xff\x00\x96\x00\xff\x00b\x00\xff\xff\xce:\x8e\x00\xff6\xfe\xff\xff\x00\x00Z\xda\xff\x00V\xff\x00v\xcej\x00j\xff\x00V\x0e\x00\xff\x00\x00\x00\xff\xe2.\x0e\x00\x00\x00\xaa\x00\xbe~\xaaj\x00\xff\xff"\xff\xe6V\xb6\xe6\x00\xba\xfab2\xff\xff\xc2\x12\xda\xff\xffb\xca\x16\xff&\xff2\x00\xff\x00\xf2\xffn&\xffRN\x00\x00Z\xff\x00\x00r\xff\x00\xff*\x00\x00\x12\xc6Z\x00\xffn\x92\xd2\xc2F\xff\xaa\xff\x06z\xff\x00fN\xd6\x00\x00b\xff\xd6\xff\x00\xff\x00\x00\xfa\x00v\x00\x00\xff\xff\x00\x1a\xa2\xff\xff\x12&\xff\xff\x92ZB\xba\xff\x00\xff\x00\xf6\xff\xd6\xea\xee\x86\x00\x00\xff\xc6\x00:\xd6\x

# Sign

## hash digest

In [ ]:
salt = secrets.token_bytes(16)
print(salt)

b'\x99@\x9a\xe8\xb6WKZ\x17\xa7\x9d\xb9\xca\xab\xddg'


In [ ]:
message = 'Hello World!'
message = message.encode('utf-8')
print(message)

b'Hello World!'


In [ ]:
if lvl == 1:
  h_digest_shake = hashlib.shake_128(message+b'\x00'+salt)
else:
  h_digest_shake = hashlib.shake_256(message+b'\x00'+salt)

In [ ]:
r=7
h_digest_bytes= h_digest_shake.digest(math.ceil(m*r/8))
print(h_digest_bytes)
print(len(h_digest_bytes))

b'\xd8w\xa4\x9b\xbaf[P\xcdE\xe6:\x85\x195a\xb6R\xae\xa7\xaa\x9a\x92Kle\xfd?(\xa0\xf4\xba^\x12\x84_\x06\xdb\x05p\t\xf2\xafu\xdd}#\x80?/'
50


In [ ]:
def int8_to_binString(intByte:int)->str:
  """Get the 8 bit representation of an integer as a binary string
     without the '0b' indicator.

  The function takes one integer byte, turns it to binary
  and returns an string containing the 8 bit representation of it.

  Parameters
  ----------
  intByte : int
    The byte, labeled as an int by python.

  Returns
  -------
  str
      A string with 8 bits to represent the integer.
  """
  binary = bin(intByte)[2:]
  return binary.zfill(8)

# Join all the bytes into a single bit-string representation.
h_digest_bits = ''.join([int8_to_binString(i) for i in h_digest_bytes])

# Take r bits sequentialy from the string m times, to create a vector within the F(2**r) field
# The bits that remain after this are discarded.
h_digest = [int(h_digest_bits[i*r:i*r+r],base=2) for i in range(m)]

print(h_digest)

[108, 29, 116, 73, 93, 105, 76, 91, 40, 51, 40, 94, 49, 106, 10, 25, 26, 88, 54, 101, 21, 58, 79, 42, 77, 36, 73, 54, 99, 23, 122, 63, 20, 40, 30, 75, 82, 120, 37, 4, 47, 65, 91, 48, 43, 64, 19, 114, 87, 93, 59, 87, 105, 14, 0, 63, 23]


## Generate v

In [ ]:
def generateV(r:int, v:int)->list:
  """Generate the vinegar variables vector.

  The function generates the closest amount of bytes to contain r*v bits in order
  to be able to get a vector of size v, with each element having r bits.

  Parameters
  ----------
  r : int
    The r paramter of the LUOV specification.

  v : int
    The v paramter of the LUOV specification.

  Returns
  -------
  list
      A list that represents the vinegar variables.
  """
  vinegar_bytes = secrets.token_bytes(math.ceil(r*v/8))

  # Join all the bytes into a single bit-string representation.
  vinegar_bits = ''.join([int8_to_binString(i) for i in vinegar_bytes])

  # Take r bits sequentialy from the string v times, to create a vector within the F(2**r) field
  # The bits that remain after this are discarded.
  vinegar = [int(vinegar_bits[i*r:i*r+r],base=2) for i in range(v)]
  return vinegar

## Build augmented matrix

In [ ]:
def buildAugmentedMatrix(C:galois.FieldArray,L:galois.FieldArray,Q1:galois.FieldArray,T:galois.FieldArray,h:galois.FieldArray,v:galois.FieldArray)->(galois.FieldArray,galois.FieldArray):
  """The implementation of the buildAugmentedMatrix algorithm of the LUOV specification.

  Parameters
  ----------
  C : galois.FieldArray
    The C vector of the LUOV specification.

  L : galois.FieldArray
    The L matrix of the LUOV specification.

  Q1 : galois.FieldArray
    The Q1 matrix of the LUOV specification.

  T : galois.FieldArray
    The T matrix of the LUOV specification.

  h : galois.FieldArray
    The hash digest to target.

  v : galois.FieldArray
    A vector containing the vinegar variables.

  Returns
  -------
  (galois.FieldArray,galois.FieldArray)
      The arrays representing the augmented matrix.
      The first being the left part and the second being the right part.
  """
  v_with_zeros = v.copy()
  v_with_zeros.resize(n)
  RHS = h - C-L@v_with_zeros
  T_with_ones = np.concatenate((T,np.identity(m)),axis=0)
  LHS = L@-T_with_ones

  GF = galois.GF(2**r)

  for k in range(m):
    Pk1=GF(findPk1(k,Q1))
    Pk2=GF(findPk2(k,Q1))
    RHS[k] = RHS[k] - v.T@Pk1@v

    Fk2=-(Pk1+Pk1.T)@T+Pk2

    LHS[k] = LHS[k] + v@Fk2
  # Here the implementation differs from the specification,
  # instead of returning the augmented matrix LHS and RHS are
  # returned separately to use numpy's solving function
  return LHS,RHS

  # return np.c_[LHS,RHS]


## Sign

In [ ]:
GF = galois.GF(2**r)
C = GF(C)
L = GF(L)
Q1 = GF(Q1)
T = GF(T)
h = GF(h_digest)

In [ ]:
no_solution = True
s_prime = np.zeros([m,1])
while no_solution:
  vinegar = GF(generateV(r=r,v=v))
  A0,A1= buildAugmentedMatrix(C,L,Q1,T,h,vinegar)

  try:
    # Try to solve the equation system given the vinegar variables
    oil = np.linalg.solve(A0,A1)
    # If we find a solution we assume it's unique and end the loop.
    no_solution = False
    # The solution (oil) is concatenated to de vinegar variables vector.
    s_prime = np.concatenate([vinegar,oil])
  except np.linalg.LinAlgError:
    # If no solution is found we stay in the loop to try with new
    # vinegar variables
    no_solution = True

In [ ]:
solution_operand_left = np.concatenate([np.identity(v),np.zeros([m,v])],axis=0)
solution_operand_right = np.concatenate([-T,np.identity(m)],axis=0)
solution_operand = np.concatenate([solution_operand_left,solution_operand_right],axis=1)
print(solution_operand)

[[1. 0. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 1. 1. 1.]
 [0. 0. 1. ... 0. 1. 1.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [ ]:
# Build S using the matrix solution_operand
s = GF(solution_operand)@s_prime
print(s)

[ 89  27  56 106  56   3 110 100  75  29  74   6 111  78   9  69  23  42
  27  33  73  27  91  55 116  94  85  68  74  86  86  63  81  25 107  71
  31   9  55  93  94  63  72 113  90  45   7  15 120  41   7   1  12  93
  31  48  74 102 102  32  49  74  20  67  52  42  46  64  57 107  91  85
 100  72 125  57  57  11 110  10 127  75  53 115  81  44  40 113   3  70
  42  37 101  18 121 104  75  47 125  43  27  37  41   8 108  27  59 125
  55  95  28  93  27 127 107  15 123  29   3  87  90  67  24  19 102  15
   4 104 106  14  13  81  66 117  70 108  87 101  58  51  29  99  78 123
  16  17  12  17 122  60  53  15 107  31  92  76 123 114  32  14  84   5
  67  20   8  93  64  32  72   0  17  90 116 110  44  11  89  82  26  25
  27  14  94  61  13  10  85   8   9  91 124 105 127  47  34 105  72  17
  56 121  74  65  62  80 105   4  95  58 108  27  35  86  33  21 122 124
  96  97 124  56  10  11  79  93  91 109  91  86  79  93 109  43  37  23
  38  64  94  94  80 106 116  40  31  81 112  45   

In [ ]:
def encode_field_element(element:galois.typing.DTypeLike,r:int)->str:
  """Get the r bit representation of an field element as a binary string
     without the '0b' indicator.

  The function takes one field element, turns it to binary
  and returns an string containing the r bit representation of it.

  Parameters
  ----------
  element : galois.typing.DTypeLike
    The field element (The number if you prefer).

  r : int
    The ammount of bits required to encode the element.

  Returns
  -------
  str
      A string with r bits to represent the field element.
  """
  return bin(element)[2:].zfill(r)

In [ ]:
def encode_siganture(signature:galois.FieldArray,salt:bytes,n:int,r:int)->bytes:
  """Encoding of the signature with the salt following the LUOV specification.

  The function takes a FieldArray, converts all its elements to binary and
  concatenates them in order, then it encodes the result into bytes (padding with
  zeros at the end to achieve the number of bytes). Finaly, the result is concatenated
  with the salt.

  Parameters
  ----------
  signature : galois.FieldArray
    The FieldArray containing the elements of the signature.

  salt : bytes
    The salt that will be added at the end of the signature.

  n : int
    The number of elements in the signature.

  r : int
    The ammount of bits of each element of the signature.

  Returns
  -------
  bytes
      The bytes of the signature and the salt concatenated.
  """
  binary_signature = ''
  # Turn all the elements in the signature vecto to an r-bit per element
  # binary string
  for i in signature:
    binary_signature += encode_field_element(i,r)
  needed_bits = n*r
  byte_signature = b''
  # Iterate through the binary string, taking 8 bits at a time to get the bytes
  # of the signature
  while needed_bits>8:
    byte_signature += int(binary_signature[:8],2).to_bytes(1,byteorder='big')
    binary_signature = binary_signature[8:]
    needed_bits -= 8
  # The remaining bits are padded with zeros at the end to make the last bytes
  if needed_bits>0:
    for i in range(8-needed_bits):
      binary_signature += '0'
    byte_signature += int(binary_signature,2).to_bytes(1,byteorder='big')
  return byte_signature + salt

In [ ]:
encoded_signature=encode_siganture(s,salt,n,r)
print(encoded_signature)
print(len(encoded_signature))

b'\xb2m\xc6\xa7\x00\xf7d\x96vPm\xf3\x84\xc5.\xa8\xda\x19&\xed\xb7\xe9z\xacIU\xab?\xa2g\\s\xe2[\xdd\xbc\xfeG\x1bKC\x8f\xf0\xa48\x11\x97O\xb0\x95\x9b2\x062\x8aCh\xa9t\x07:\xed\xd5\xc9#\xeb\x97"\xf7\n\xff-\xaf:+\x14q\x07\x19R\\\xa4\xbc\xe8\x96\xbf\xea\xb3iT\x88\xd8m\xdf\xd6\xf7\xce]7\xffX\xffgA\xd7\xb5\x0c\xc1<\xc3\xc2h\xd48m\x18]cl\xaf\x95\xd33\xb8\xe7{ Da\x1fO\x1a\x8f\xd6~\xe4\xcf|\x90\x0e\xa8\x16\x19A\x17` \x90\x00\x8d\xae\x9b\x96\x0b\xb3H\xd1\x93c\xaf=\x1a*\xa8\x816\xfei\xfe\xbd\x16\x99\x04\\y\x95\x05\xf5\r!/\xba\xd8m\x1dd%}|\xc1\x87\xe3\x81B\xe7\xdd\xb7\xb6\xddi\xf7v\xabJ]4\x0b\xd7\xa8j\xe8\xa0\xfd\x1e\x0b@0\xb4RBY\x9f@\x99@\x9a\xe8\xb6WKZ\x17\xa7\x9d\xb9\xca\xab\xddg'
239
